In [1]:
# NOT WORKING: need to fix forward (unconditional generation)

In [2]:
import os
import sys
sparse_multiview = f'{os.getcwd()}/../sparse_multiview'
sys.path.append(sparse_multiview)
from multiview import *
from omegaconf import OmegaConf
import torch
import matplotlib.pyplot as plt
args = OmegaConf.load(f'{sparse_multiview}/configs/views5.yaml')

SyntaxError: invalid syntax (multiview.py, line 175)

In [ ]:
model = MultiViewDiffusionModel(args).cuda()
model.eval()
dataset = MultiViewDataset(args)
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=1,
    shuffle=True,
    #pin_memory=True
)

In [ ]:
batch = next(iter(dataloader))
print(batch['target'].shape, batch['source'].shape)
target = batch['target'].squeeze(0).permute((1, 2, 0))
source = batch['source'].squeeze(0).permute((1, 2, 0))
fig, axs = plt.subplots(1, 4)
axs[0].imshow((target+1)/2)
axs[0].imshow((source+1)/2)
axs[1].imshow(batch['depth'].squeeze())
axs[2].imshow(batch['mask'].squeeze())
plt.show()

In [ ]:
# batch = next(iter(dataloader))
# image = batch['image'].squeeze(0).permute((1, 2, 0))
# views = [img.permute((1, 2, 0)) for img in batch['views'].squeeze(0)]
# fig, axs = plt.subplots(1, 3)
# axs[0].imshow((image+1)/2)
# axs[1].imshow(batch['depth'].squeeze())
# axs[2].imshow(batch['mask'].squeeze())
# plt.show()
# fig, axs = plt.subplots(1, len(views), figsize=(40,40))
# for i, view in enumerate(views):
#     axs[i].imshow(view)
#     axs[i].set_axis_off()
# plt.show()

In [ ]:
batch['depth']

In [3]:
images = model.forward({k:v.cuda() for k,v in batch.items()})
images[0]

NameError: name 'model' is not defined

In [ ]:
model.multiview_encoder.mae_image_mask_ratio = 0
images, attention_maps = model.forward_with_crossattention({k:v.cuda() for k,v in batch.items()})
attention_maps = attention_maps.detach().cpu()
orig_image = images[0]
orig_image

In [ ]:
dataset.num_views_per_inst

In [ ]:
# Inspired by 
# https://github.com/AttendAndExcite/Attend-and-Excite/blob/main/notebooks/explain.ipynb
# https://github.com/AttendAndExcite/Attend-and-Excite/blob/main/utils/vis_utils.py 
# https://github.com/AttendAndExcite/Attend-and-Excite/blob/main/utils/ptp_utils.py

import ptp_utils
from vis_utils import show_image_relevance
# show spatial attention for indices of tokens to strengthen
n_tokens = attention_maps.shape[2]
res = 16

mae = model.multiview_encoder.model
m = n_tokens // dataset.num_views_per_inst

context = batch['views']
context = context.view(-1, *context.shape[2:])
context_tokens = mae.patchify(context.cuda()).reshape(-1, 768).cpu().numpy()
non_white = np.abs(context_tokens-1).mean(axis=1)
context_tokens = np.array([context_tokens[i] for i in range(len(context_tokens)) if non_white[i] >= 1e-2])

for view in range(dataset.num_views_per_inst):
    images = []
    for i in range(view*m, view*m+m):
        image = attention_maps[:, :, i]
        image = show_image_relevance(image, orig_image)
        image = image.astype(np.uint8)
        image = np.array(Image.fromarray(image).resize((res ** 2, res ** 2)))
        images.append(image)
    ptp_utils.view_images(np.stack(images[1:11], axis=0))
    plt.show()
    
    imgs = context_tokens.reshape(-1, 3, 16, 16).transpose((0,2,3,1))
    #imgs = context_tokens[:10].reshape(-1, 16, 16, 3)
    #print(imgs[0])
    fig, axs = plt.subplots(1, len(imgs))
    for i in range(len(imgs)):
        axs[i].imshow(imgs[i])
        axs[i].set_axis_off()
    plt.show()